In [23]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

In [24]:
# Explore the "covid_infected_deaths" table
total_covid_infected_deaths_perDistrict = pd.read_stata("../covid_data/covid/covid_infected_deaths.dta")
total_covid_infected_deaths_perDistrict = total_covid_infected_deaths_perDistrict[total_covid_infected_deaths_perDistrict["lgd_state_name"].isin(["tamil nadu", "madhya pradesh", "maharashtra", "uttar pradesh", "kerala", "bihar"])]
total_covid_infected_deaths_perDistrict.head()

,lgd_state_id,lgd_district_id,date,lgd_state_name,lgd_district_name,total_cases,total_deaths
34884,10,188,2020-01-30,bihar,araria,NaN,NaN
34885,10,188,2020-02-02,bihar,araria,NaN,NaN
34886,10,188,2020-02-03,bihar,araria,NaN,NaN
34887,10,188,2020-03-02,bihar,araria,NaN,NaN
34888,10,188,2020-03-03,bihar,araria,NaN,NaN


In [25]:
# Explore the "covid_infected_deaths" table
total_covid_infected_deaths_perDistrict["year"] = total_covid_infected_deaths_perDistrict["date"].dt.year
total_covid_infected_deaths_perDistrict["month"] = total_covid_infected_deaths_perDistrict["date"].dt.month
total_covid_infected_deaths_perDistrict.drop(["date"], axis=1, inplace=True)

# Explore the "total_covid_infected_deaths_perDistrict" table
total_covid_infected_deaths_perDistrict.info()

<class 'pandas.core.frame.DataFrame'>
Index: 157896 entries, 34884 to 416159
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   lgd_state_id       157896 non-null  object 
 1   lgd_district_id    157896 non-null  object 
 2   lgd_state_name     157896 non-null  object 
 3   lgd_district_name  157896 non-null  object 
 4   total_cases        153014 non-null  float64
 5   total_deaths       153014 non-null  float64
 6   year               157896 non-null  int32  
 7   month              157896 non-null  int32  
dtypes: float64(2), int32(2), object(4)
memory usage: 9.6+ MB


In [26]:
# Get the monthly cumulative cases and deaths in district level
yearly_cumulative = total_covid_infected_deaths_perDistrict.groupby(["lgd_state_id", "year"]).agg({
    "total_cases": "max",
    "total_deaths": "max"
}).reset_index()

# Get the monthly new cases and deaths in state level
yearly_new_cases_deaths = yearly_cumulative.groupby(["lgd_state_id"]).diff().fillna(0)

# Merge the above data
yearly_cumulative[["new_cases", "new_deaths"]] = yearly_new_cases_deaths[["total_cases", "total_deaths"]]

# Clean the "monthly_cumulative" table
yearly_cumulative.replace("", np.nan, inplace=True)
yearly_cumulative = yearly_cumulative.dropna()

yearly_cumulative

,lgd_state_id,year,total_cases,total_deaths,new_cases,new_deaths
0,09,2020,78257.0,1111.0,0.0,0.0
1,09,2021,238839.0,2651.0,160582.0,1540.0
2,10,2020,48910.0,380.0,0.0,0.0
3,10,2021,147007.0,2334.0,98097.0,1954.0
4,23,2020,54918.0,873.0,0.0,0.0
5,23,2021,153230.0,1665.0,98312.0,792.0
6,27,2020,372655.0,11116.0,0.0,0.0
7,27,2021,1154776.0,19594.0,782121.0,8478.0
8,32,2020,90820.0,653.0,0.0,0.0
9,32,2021,602800.0,4996.0,511980.0,4343.0


In [27]:
# Get shapefile in state level
state_shp = gpd.read_file("./output_shpfile/state_key_with_polygon/state_key_with_polygon.shp")

# Check the "district_shp" table
state_shp.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   lgd_sta_id  36 non-null     object  
 1   lgd_sta_na  36 non-null     object  
 2   pc11_st_id  36 non-null     object  
 3   pc11_st_na  36 non-null     object  
 4   geometry    36 non-null     geometry
dtypes: geometry(1), object(4)
memory usage: 1.5+ KB


In [28]:
# Merge the above two tables (2018)
covid_total_mortality_perState_perYear_shp = pd.merge(state_shp, yearly_cumulative, left_on="lgd_sta_id", right_on="lgd_state_id", how="left")
covid_total_mortality_perState_perYear_shp = covid_total_mortality_perState_perYear_shp.dropna()

# Check the "covid_total_mortality_perState_perYear_shp" table
covid_total_mortality_perState_perYear_shp.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 12 entries, 7 to 37
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   lgd_sta_id    12 non-null     object  
 1   lgd_sta_na    12 non-null     object  
 2   pc11_st_id    12 non-null     object  
 3   pc11_st_na    12 non-null     object  
 4   geometry      12 non-null     geometry
 5   lgd_state_id  12 non-null     object  
 6   year          12 non-null     float64 
 7   total_cases   12 non-null     float64 
 8   total_deaths  12 non-null     float64 
 9   new_cases     12 non-null     float64 
 10  new_deaths    12 non-null     float64 
dtypes: float64(5), geometry(1), object(5)
memory usage: 1.1+ KB


In [29]:
covid_total_mortality_perState_perYear_shp.drop(columns=["lgd_state_id"], inplace=True)
covid_total_mortality_perState_perYear_shp.rename(columns={"total_cases": "total_case",
                                                           "total_deaths": "total_deat"}, inplace=True)

# Check the "mortality_perDistrict_perMonth_shp" table
covid_total_mortality_perState_perYear_shp.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 12 entries, 7 to 37
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   lgd_sta_id  12 non-null     object  
 1   lgd_sta_na  12 non-null     object  
 2   pc11_st_id  12 non-null     object  
 3   pc11_st_na  12 non-null     object  
 4   geometry    12 non-null     geometry
 5   year        12 non-null     float64 
 6   total_case  12 non-null     float64 
 7   total_deat  12 non-null     float64 
 8   new_cases   12 non-null     float64 
 9   new_deaths  12 non-null     float64 
dtypes: float64(5), geometry(1), object(4)
memory usage: 1.0+ KB


In [30]:
# Export the "mortality_perDistrict_perMonth_shp" table to shapefile
os.mkdir("./output_shpfile/covid_total_mortality_perState_perYear_shp")
covid_total_mortality_perState_perYear_shp.to_file(
    "./output_shpfile/covid_total_mortality_perState_perYear_shp/covid_total_mortality_perState_perYear_shp.shp", driver="ESRI Shapefile")